Hi Kagglers!
I tried Data Analsys & fitting LightLGB.
ref:https://www.kaggle.com/eduardogutierrez/tps-nov-21-exploratory-data-analysis

I will try to tune the parameters.
If you have any other techniques or advice, Please discuss with me.
Thank you for Reading! :D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score
#Data Read
df=pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
df.describe().T.style.background_gradient(cmap = 'Blues')\
                          .bar(subset = ["mean",], color = 'lightgreen')\
                           .bar(subset = ["std"], color = '#ee1f5f')

In [ ]:
fig, axes = plt.subplots(10,10,figsize = (20, 15))
axes = axes.flatten()
for idx, ax in enumerate(axes):
    sns.kdeplot(data = df, x = f'f{idx}', fill = True, ax = ax)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_xlabel(''); ax.set_ylabel('')
    ax.set_title(f'f{idx}', loc = 'right', fontsize = 12)
fig.tight_layout()
plt.show()

In [ ]:
#def color_background_lightgreen(val):
#    color = 'lightgreen' if val > 1.5 else ''
#    return 'background-color: %s' % color

#表示
#df.style.applymap(color_background_lightgreen)

#LightLGB

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [ ]:
date_test=pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")
date_train=pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")


In [ ]:
NG_List=[]
for i in range(100):
    arr_std=date_train["f"+str(i)].std()
    if(arr_std < 1.5) :
        NG_List.append("f"+str(i))
print(NG_List)

In [ ]:
#Data_Split
df_train,df_val=train_test_split(date_train,test_size=0.3)

#Training Data
X_train=df_train.drop("target",axis=1)
X_train=X_train.drop("id",axis=1)
y_train=df_train["target"].values
X_train_df=X_train.drop(NG_List,axis=1)
X_train_df.head()
#Val Data
X_val=df_val.drop("target",axis=1)
X_val=X_val.drop("id",axis=1)
y_val=df_val["target"].values
X_val_df=X_val.drop(NG_List,axis=1)
X_val_df.head()

In [ ]:
#LGB Modeling
params = {
    "objective" : "binary",
    "random_state":100,
    "verbose":0,
    "metrics" : "auc"
}


train_d=lgb.Dataset(X_train_df,y_train)
val_d=lgb.Dataset(X_val_df,y_val)

model=lgb.train(params,train_d,valid_sets=val_d,num_boost_round=100,early_stopping_rounds=100)

In [ ]:
#fitting
data_X=date_test.drop(NG_List,axis=1)
data_X=data_X.drop("id",axis=1)

pred=model.predict(data_X)
pred = (pred > 0.4) * 1


In [ ]:
#submit
PassengerId=date_test["id"]
print(PassengerId.shape)
print(PassengerId.head())
Predictions=pd.DataFrame(data=pred,index=PassengerId,columns=['Target'])
Predictions.to_csv("submission.csv")